<h1 align="center"> <span style="font-size: 20px;"> Proyecto Final Grupo Nº 7 </span> <br>
    <align="center"> <span style="font-size: 15px;">  Proyecto Yelp & Google Maps - Reseñas y Recomendaciones</span><br>         
<align="center"> <span style="font-size: 15px;">@utores:  Javier Castro, Luca Ramallo, Luis Ramírez, Lesmen García.</span> <br>
  <a href="https://github.com/lesmengp/Proyecto-Final-Grupo-07.git">GitHub: <span style="font-size: 20px;">Proyecto Yelp & Google Maps</span></a> 
</h1>

# ETL en GCP

### Librerías a Utilizar:

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import os
import io
import json
import gcsfs


### Ubicación de los ETL en GCP: 
ruta_etl_datalike= "gs://gmy/etl/"

# ETL de los siguientes Datasets de Yelp: 
### 1) yelp_user
### 2) yelp_tip
### 3) yelp_business
### 4) yelp_review

### 1) ETL del Dataset: yelp_user

El DataSets **'yelp_user'** ocupa un espacio aproximado de 2.8 GB con un historial de 20 años.<Br>
Por limitaciones de recursos, se realizaron las siguientes acciones locales en Python, para poder cargar en Google Cloud: <Br>
1. Se realiza un filtro de los últimos 5 años.
2. Se divide el dataset en 7 archivos para facilitar la carga a Google Cloud
3. La carga inicial a Google Cloud se realiza en el DataLike: gs://gmy/dataset

Consolidando los 7 Datasets de 'yelp_user' en uno solo Dataset.

### Cargando y Concatenando los Datasets de 'yelp_users'

In [13]:
### Concatenando los 7 Dataset en uno solo

# Nombre del bucket en Cloud Storage
bucket_name = "gmy"

# Lista para almacenar los DataFrames de cada archivo
dfs = []

# Crea un sistema de archivos de Google Cloud Storage (GCS)
fs = gcsfs.GCSFileSystem(project="Proyecto Final - Henry")

# Itera sobre los archivos numerados del 0 al 6
for i in range(7):
    # Construye la ruta completa del archivo Parquet
    archivo_parquet = f"gs://{bucket_name}/dataset/user/user-reducido_{i}.parquet"
    
    # Lee el archivo Parquet y carga los datos en un DataFrame de pandas
    with fs.open(archivo_parquet, "rb") as f:
        df = pd.read_parquet(f)
    
    # Agrega el DataFrame a la lista
    dfs.append(df)

# Concatena los DataFrames en uno solo
df_final = pd.concat(dfs, ignore_index=True)
df_final

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,RDlBwvQe5oEQ8STtj6a5rg,Dee,4,2018-11-06,5,0,2,,"FrLfQoTXb7k68ZqBQTHSew, 88wN6ysc8Z1LSgaa-Au2MA...",0,...,0,0,0,0,0,0,0,0,0,0
1,wZ5mQSbZR5UpGpnddGT7mw,Odalis,178,2018-04-15,542,95,429,"2018,2019,20,20,2021","zGAGITTaxhi2J14604F_iA, WoMHyzfYFI8juti3DbaIvg...",22,...,3,0,0,0,21,29,33,33,21,26
2,Oj8Ohm_2OlV6hjKMsQn69w,Emily,2,2018-07-01,1,0,0,,"YQhTRxurCw9uNe3stFyHHA, XUbfn0nkZgcsLixJJ_2WKQ...",0,...,0,0,0,0,0,0,0,0,0,0
3,LRjvkbcZ5S4efH0xAVfkxw,Maggie,1,2018-09-08,2,0,0,,"VNHx2MZANbDBZml5eNMP2g, 5CmrzlkGIglc5A-4b4S8Ig...",0,...,0,0,0,0,0,0,0,0,0,0
4,705syU_EmyN_hnlnmYy3nQ,Ella,2,2018-03-13,1,0,0,,"Ow2zUmCxcyOMd0ulTM30Zw, BkrImxMimcTtwNBC4NwZyQ...",0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329169,2anBQswThYJsb-d9jsEMOA,Mackenzie,58,2018-03-18,69,8,51,"2018,2019,20,20,2021","TWDmrDQgda42KxsAIHt7rg, Eq7uvwdFebvUzmXKN4W_Kw...",4,...,0,2,0,0,20,1,7,7,0,0
329170,vXFC3SDX-EluGtkO--W4fg,Mackenzie,4,2018-04-15,1,0,1,,"1-vtu9ZSAD78OXtVy4Ocdg, -lvQdF-0veHI3ZqopS-_yg...",0,...,1,0,0,0,0,0,2,2,0,2
329171,LBUXcNlBV0oLvRp7zj76Hg,Nancy,2,2018-05-09,0,0,0,,"B3YIazSEBZrw1iIlIZMaQA, uvG_WW06Fs5vTKeF9gOQ0A...",0,...,0,0,0,0,0,0,0,0,0,0
329172,K47RXWydxGWGQOJfFViPaA,Jason,21,2018-03-28,6,0,18,,"s4Z-9JpvIEEH4nHrqGH70g, Rll1W4rrlucwqDmH8gXUJQ...",1,...,0,0,0,0,0,1,4,4,1,0


In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329174 entries, 0 to 329173
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             329174 non-null  object 
 1   name                329174 non-null  object 
 2   review_count        329174 non-null  int64  
 3   yelping_since       329174 non-null  object 
 4   useful              329174 non-null  int64  
 5   funny               329174 non-null  int64  
 6   cool                329174 non-null  int64  
 7   elite               329174 non-null  object 
 8   friends             329174 non-null  object 
 9   fans                329174 non-null  int64  
 10  average_stars       329174 non-null  float64
 11  compliment_hot      329174 non-null  int64  
 12  compliment_more     329174 non-null  int64  
 13  compliment_profile  329174 non-null  int64  
 14  compliment_cute     329174 non-null  int64  
 15  compliment_list     329174 non-nul

In [12]:
### Valores en Blanco de la columna 'elite'
df_final[df_final['elite'] == ""].count()

user_id               325033
name                  325033
review_count          325033
yelping_since         325033
useful                325033
funny                 325033
cool                  325033
elite                 325033
friends               325033
fans                  325033
average_stars         325033
compliment_hot        325033
compliment_more       325033
compliment_profile    325033
compliment_cute       325033
compliment_list       325033
compliment_note       325033
compliment_plain      325033
compliment_cool       325033
compliment_funny      325033
compliment_writer     325033
compliment_photos     325033
dtype: int64

La columna 'elite' tiene muchos valores en blanco. Pero se analizará en el EDA

### Eliminando Columnas del Dataset YELP USER

In [13]:
### Validando Columnas del Dataset
df_final.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [14]:
### Eliminando Columnas del Dataset YELP USER

# Creando Variable con las Columnas a Eliminar
Columnas_to_Eliminar = ['name', 'funny', 'friends', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos']

### Elimina las columnas especificadas del DataFrame
df_final = df_final.drop(Columnas_to_Eliminar, axis=1)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329174 entries, 0 to 329173
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   user_id        329174 non-null  object 
 1   review_count   329174 non-null  int64  
 2   yelping_since  329174 non-null  object 
 3   useful         329174 non-null  int64  
 4   cool           329174 non-null  int64  
 5   elite          329174 non-null  object 
 6   fans           329174 non-null  int64  
 7   average_stars  329174 non-null  float64
dtypes: float64(1), int64(4), object(3)
memory usage: 20.1+ MB


### Convertiendo la columna 'yelping_since' en formato fecha

In [15]:
### Convertiendo la columna 'yelping_since' en formato fecha
df_final['yelping_since'] = pd.to_datetime(df_final['yelping_since'], format='%Y-%m-%d', errors='coerce')
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329174 entries, 0 to 329173
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   user_id        329174 non-null  object        
 1   review_count   329174 non-null  int64         
 2   yelping_since  329174 non-null  datetime64[ns]
 3   useful         329174 non-null  int64         
 4   cool           329174 non-null  int64         
 5   elite          329174 non-null  object        
 6   fans           329174 non-null  int64         
 7   average_stars  329174 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 20.1+ MB


In [16]:
### Ordenar Dataset por Fecha
# Ordena el DataFrame por la columna "fecha" en orden ascendente
df_final = df_final.sort_values(by="yelping_since", ascending=True)
df_final

,user_id,review_count,yelping_since,useful,cool,elite,fans,average_stars
172698,TIvO4GClIYXzxu_jVZ7ruQ,2,2018-01-01,2,0,,0,5.00
197601,Bxz-sFWLiPx-rR99eBr6TQ,1,2018-01-01,7,1,,0,1.00
238402,-t7FxAVyh_2XQvi_PvVQkg,1,2018-01-01,0,0,,0,2.00
80086,9JZLK2tiLUsFLD3wZdUIWQ,3,2018-01-01,2,1,,0,1.33
46652,J7Y_oLaopwipWnTR8OgVEA,1,2018-01-01,0,0,,0,5.00
...,...,...,...,...,...,...,...,...
321060,IaM7TRiBmwrgezg4ppBqpA,1,2022-01-19,0,0,,0,1.00
321793,8f64yYInvWEzGnxEXgprzg,1,2022-01-19,0,0,,0,5.00
302670,6uzx3jSpkuSA8gOk4bXAWw,1,2022-01-19,0,0,,0,5.00
54877,jNwnKn1LVsDZrLB9YIxsog,1,2022-01-19,0,0,,0,1.00


### Guardar Dataset Unificado de 'yelp_users'

In [17]:
### Guardar Dataset Final de yelp_user

# Ruta completa en Cloud Storage para guardar el archivo en formato CSV
ruta_etl_datalike = "gs://gmy/etl/yelp_users.csv"

# Guarda el DataFrame en formato CSV en Cloud Storage
df_final.to_csv(ruta_etl_datalike, index=False)


### 2) ETL del Dataset: yelp_tip

Todo el ETL del Dataset 'yelp_tip' se realizo localmente. Solo se procederá a ubicarlo en su Dataliake correspondiente en formato .csv para su posterior análisis del EDA.

In [18]:
### Trabajando con TIP
# Ruta completa en Cloud Storage
ruta_etl_local = "gs://gmy/dataset/tip/tip-reducido.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
df_tip = pd.read_parquet(ruta_etl_local, storage_options={"project": "Proyecto Final - Henry"})
df_tip

,user_id,business_id,text,date,compliment_count
0,1viszljzsa8W7VnYPuWosw,Zg83olSjsdXAhE5EEy5IcQ,Order crispy and they bake it up good.,2018-04-01,0
1,YnlCpuaBa3qWBp4te8pGmA,XIKYdKWq72zUYsq8NBxcCQ,The honey glazed salmon is amazing!,2018-01-14,0
2,XJDLaoN1PerKw2woiKeepA,t0zwddmbOGQOADrAxEPHQQ,Come by for some Jelly Bean sours this weekend!,2018-03-31,0
3,9M_nk0kA3CTJt6UoxRAK_g,oJf3IZloLf_PDeEeZW5nDg,Meatball sandwich,2018-03-27,0
4,0LY6l9-Z2IrVtnfj6OX06w,Zmwm6d872C8kWJr4b6UGfg,Great tasty food,2018-04-01,0
...,...,...,...,...,...
193028,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11,0
193029,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30,0
193030,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05,0
193031,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20,0


In [19]:
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193033 entries, 0 to 193032
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           193033 non-null  object
 1   business_id       193033 non-null  object
 2   text              193033 non-null  object
 3   date              193033 non-null  object
 4   compliment_count  193033 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 7.4+ MB


### Convertiendo la columna 'date' en formato fecha

In [20]:
### Convertiendo la columna 'date' en formato fecha
df_tip['date'] = pd.to_datetime(df_tip['date'], format='%Y-%m-%d', errors='coerce')

In [21]:
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193033 entries, 0 to 193032
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           193033 non-null  object        
 1   business_id       193033 non-null  object        
 2   text              193033 non-null  object        
 3   date              193033 non-null  datetime64[ns]
 4   compliment_count  193033 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 7.4+ MB


### Guardar ETL del Dataset: yelp_tip

In [22]:
### Guardar Dataset Final de TIP

# Ruta completa en Cloud Storage para guardar el archivo CSV
ruta_etl_datalike = "gs://gmy/etl/yelp_tips.csv"

# Guarda el DataFrame en formato CSV en Cloud Storage
df_tip.to_csv(ruta_etl_datalike, index=False)

### 3) ETL del Dataset: yelp_business

In [27]:
### Valida Primer Dataframe
# Ruta completa en Cloud Storage
ruta_etl_local = "gs://gmy/dataset/business/business.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
yelp_business = pd.read_parquet(ruta_etl_local, storage_options={"project": "Proyecto Final - Henry"})
yelp_business.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
1,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471658,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
2,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,AZ,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
3,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,PA,63144,38.627695,-90.340465,2.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."
4,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,PA,33707,27.766590,-82.732983,3.5,5,1,None,"Synagogues, Religious Organizations","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."


In [28]:
yelp_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145216 entries, 0 to 145215
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   145216 non-null  object 
 1   name          145216 non-null  object 
 2   address       145216 non-null  object 
 3   city          145216 non-null  object 
 4   state         145216 non-null  object 
 5   postal_code   145199 non-null  object 
 6   latitude      145216 non-null  float64
 7   longitude     145216 non-null  float64
 8   stars         145216 non-null  float64
 9   review_count  145216 non-null  int64  
 10  is_open       145216 non-null  int64  
 11  attributes    132343 non-null  object 
 12  categories    145114 non-null  object 
 13  hours         123104 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 15.5+ MB


In [29]:
yelp_business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [30]:
### Eliminando Columnas del Dataset YELP USER

# Creando Variable con las Columnas a Eliminar
Columnas_to_Eliminar = ['postal_code', 'is_open', 'hours']

### Elimina las columnas especificadas del DataFrame
yelp_business = yelp_business.drop(Columnas_to_Eliminar, axis=1)

yelp_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145216 entries, 0 to 145215
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   145216 non-null  object 
 1   name          145216 non-null  object 
 2   address       145216 non-null  object 
 3   city          145216 non-null  object 
 4   state         145216 non-null  object 
 5   latitude      145216 non-null  float64
 6   longitude     145216 non-null  float64
 7   stars         145216 non-null  float64
 8   review_count  145216 non-null  int64  
 9   attributes    132343 non-null  object 
 10  categories    145114 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 12.2+ MB


In [32]:
yelp_business.head(5)

,business_id,name,address,city,state,latitude,longitude,stars,review_count,attributes,categories
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
1,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,40.338183,-75.471658,4.5,13,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food"
2,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,AZ,36.269593,-87.058943,2.0,6,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
3,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,PA,38.627695,-90.340465,2.5,13,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Sporting Goods, Fashion, Shoe Stores, Shopping..."
4,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,PA,27.766590,-82.732983,3.5,5,None,"Synagogues, Religious Organizations"


In [33]:
yelp_business.isna().sum()

business_id         0
name                0
address             0
city                0
state               0
latitude            0
longitude           0
stars               0
review_count        0
attributes      12873
categories        102
dtype: int64

In [35]:
### Información Detallada de la Columna: 'attributes'
DataSets = ['yelp_business']
Columna = 'attributes'

for dataset in DataSets:
    df = globals()[dataset]
    print(f"Información del DataSets: '{','.join(DataSets)}'")
    print(f"Información Detallada de la Columna: {Columna}")
    print("="*80)
    print(f"Analizando los Valores Nulos en la Columna:   '{Columna}'")
    print(f"Total Registros: {df.shape[0]}")
    print(f"Total Valores Nulos: {df[Columna].isna().sum()}")
    print(f"Porcentaje Valores Nulos: {round((df[Columna].isna().sum()/df.shape[0])*100,3)} %")
    #print(f"Valor/es de la Moda:\n{df[Columna].mode()[0]}")
    #print(f"Estadistica Descriptiva de la Columna {Columna}:\n{df[Columna].describe()}")
    print("="*80)

Información del DataSets: 'yelp_business'
Información Detallada de la Columna: attributes
Analizando los Valores Nulos en la Columna:   'attributes'
Total Registros: 145216
Total Valores Nulos: 12873
Porcentaje Valores Nulos: 8.865 %


### Guardar Dataset Final de yelp_business

In [37]:
### Guardar Dataset Final de TIP

# Ruta completa en Cloud Storage para guardar el archivo CSV
ruta_etl_datalaike = "gs://gmy/etl/yelp_business.csv"

# Guarda el DataFrame en formato CSV en Cloud Storage
yelp_business.to_csv(ruta_etl_datalaike, index=False)

### 4) ETL del Dataset: yelp_reviews

### Cargando y Concatenando los Datasets de 'yelp_reviews'

Esta Dataset de 'yelp_review' esta filtrado por los últimos 5 años

In [5]:
### Concatenando los 28 Dataset de yelp_reviews en uno solo

# Nombre del bucket en Cloud Storage
bucket_name = "gmy"

# Lista para almacenar los DataFrames de cada archivo
dfs = []

# Crea un sistema de archivos de Google Cloud Storage (GCS)
fs = gcsfs.GCSFileSystem(project="Proyecto Final - Henry")

# Itera sobre los archivos numerados del 0 al 27
for i in range(7):
    # Construye la ruta completa del archivo Parquet
    archivo_parquet = f"gs://{bucket_name}/dataset/review/review-reducido_{i}.parquet"
    
    # Lee el archivo Parquet y carga los datos en un DataFrame de pandas
    with fs.open(archivo_parquet, "rb") as f:
        df = pd.read_parquet(f)
    
    # Agrega el DataFrame a la lista
    dfs.append(df)

# Concatena los DataFrames en uno solo
df_final = pd.concat(dfs, ignore_index=True)
df_final

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07
1,lUUhg8ltDsUZ9h0xnwY4Dg,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,4,1,0,1,I was really between 3 and 4 stars for this on...,2018-07-17
2,JBWZmBy69VMggxj3eYn17Q,aFa96pz67TwOFu4Weq5Agg,kq5Ghhh14r-eCxlVmlyd8w,5,0,0,0,My boyfriend and I tried this deli for the fir...,2018-08-23
3,E9AB7V4z8xrt2uPF7T55FQ,iYY5Ii1LGpZCpXFkHlMefw,Zx7n8mdt8OzLRXVzolXNhQ,5,0,0,0,Amazing biscuits and (fill in the blank). Grea...,2018-04-27
4,A4n4YaE-owOVgTQcrVqHUw,S7bjj-L07JuRr-tpX1UZLw,I6L0Zxi5Ww0zEWSAVgngeQ,4,0,0,0,The cafe was extremely cute. We came at 8am an...,2018-07-07
...,...,...,...,...,...,...,...,...,...
699995,hhl8Nozr0tADgVjtX3JlVw,e7xUNW5THOyDrni0W662Zw,6KJTlLbdMH4jdDrYHZNv6w,4,1,0,1,This was the second time we came to Precinct p...,2018-03-29
699996,LohgzhBttHyNv_D2Z_uB1g,Kh9mzyE3bRUAWCMyOVn53g,5FnHWI9_ffeLxPZXWYULjA,5,0,0,0,Just spent the weekend in Nashville for our ho...,2019-03-11
699997,Wj9aENIJf2a7dUMs-2kMQQ,8ccg7g0R0JXhjjta1ZzEfA,-6jVwUuXC1-dyr2YRCEU6Q,5,0,1,0,best Indian food in St. Louis there is nothing...,2019-06-20
699998,_Crlj7DAPJWDVN2Ti4c47w,Pz25TQRkx6C_8sh8WySICA,ntu7zZ4myxSVDUlpwK26aA,1,0,0,0,The rice was super hard could possibly chip a ...,2019-07-12


In [6]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   review_id    700000 non-null  object
 1   user_id      700000 non-null  object
 2   business_id  700000 non-null  object
 3   stars        700000 non-null  int64 
 4   useful       700000 non-null  int64 
 5   funny        700000 non-null  int64 
 6   cool         700000 non-null  int64 
 7   text         700000 non-null  object
 8   date         700000 non-null  object
dtypes: int64(4), object(5)
memory usage: 48.1+ MB


### ### Convertiendo la columna 'date' a tipo fecha

In [11]:
### Convertiendo la columna 'date' a tipo fecha
df_final['date'] = pd.to_datetime(df_final['date'], errors='coerce')
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   review_id    700000 non-null  object        
 1   user_id      700000 non-null  object        
 2   business_id  700000 non-null  object        
 3   stars        700000 non-null  int64         
 4   useful       700000 non-null  int64         
 5   funny        700000 non-null  int64         
 6   cool         700000 non-null  int64         
 7   text         700000 non-null  object        
 8   date         700000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 48.1+ MB


In [10]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   review_id    700000 non-null  object
 1   user_id      700000 non-null  object
 2   business_id  700000 non-null  object
 3   stars        700000 non-null  int64 
 4   useful       700000 non-null  int64 
 5   funny        700000 non-null  int64 
 6   cool         700000 non-null  int64 
 7   text         700000 non-null  object
 8   date         700000 non-null  object
dtypes: int64(4), object(5)
memory usage: 48.1+ MB


In [12]:
df_final.isna().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [ ]:
df_final.info()

### Guardar Dataset Final de 'yelp_reviews'

In [13]:
### Guardar Dataset Final de Yelp_reviews

# Ruta completa en Cloud Storage para guardar el archivo CSV
ruta_etl_datalaike = "gs://gmy/etl/yelp_reviews.csv"

# Guarda el DataFrame en formato CSV en Cloud Storage
df_final.to_csv(ruta_etl_datalaike, index=False)

# Fin ETL en GCP